### Анализ датасета __who_suicide_statictics__.

### Выполнил __Сафиуллин Артур Джамилевич, группа ПМИ-2017-Б-Д-01__.

## Задание
* Расчитать основные описательные характеристики.
* При необходимости почистить данные.
* Провести визуальный анализ.
* Выдвинуть 3-6 гипотез относительно качественных и количественных признаков и доказать их.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Загрузим данные в фрейм данных Pandas.

In [ ]:
data = pd.read_csv('/kaggle/input/who-suicide-statistics/who_suicide_statistics.csv')

## Описательные характеристики.

За __количественные__ признаки возьмем
* Население (population)
* Кол-во суицидов (suicides_no)

За __качественные__ признаки возьмем
* Возрастная группа (age)
* Год (year)
* Пол (sex)
* Страна (country)

Характеристики количественных данных.

In [ ]:
print("Медиана для суицидов: ", data['suicides_no'].median())
print("Среднеквадратическое отклонение для суицидов: ", data['suicides_no'].std())
print("Дисперсия для суицидов: ", data['suicides_no'].var())
print("Мода для суицидов: ", data['suicides_no'].mode())

print("Медиана для популяции: ", data['population'].median())
print("Среднеквадратическое отклонение для популяции: ", data['population'].std())
print("Дисперсия для популяции: ", data['population'].var())
print("Мода для популяции: ", data['population'].mode())

In [ ]:
data.astype('object').describe()

Как мы видим, некоторые данные выдают некорректные значения, так как не заполнены. Давайте выясним сколько в этом датасете таких данных и избавимся от них.

In [ ]:
na_stats = pd.DataFrame([],columns=['count_na','prop_na'])
na_stats['count_na'] = (data.isna().sum())
na_stats['prop_na'] = (data.isna().sum()/data.shape[0])
na_stats

Около 5% данных по суицидам и 12% по населения не заполнены.

## Чистка данных.

In [ ]:
data = data.dropna()

Проверим первые 10 элементов на наличие пустых данных.

In [ ]:
data.head()

Как мы можем заметить, первые 24 строки были отброшены, значит данные были почищены.

## Визуализация данных и анализ.

Взглянем как распределено кол-во суицидов по годам на графике.

In [ ]:
data.groupby("year").sum().sort_values(by='year',ascending=True)[['suicides_no']].plot(kind='bar',figsize=(16,8),title='Распределение числа суицидов по годам')

Взглянем на топ-10 стран по кол-ву суицидов за все время.

In [ ]:
data[['country','suicides_no']].groupby(['country']).agg('sum').sort_values(by = 'suicides_no',ascending = False).head(10)

Построим график

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)

xval = pd.DataFrame(data.groupby(['country'])['suicides_no'].sum().reset_index())
xval.sort_values(by = ['suicides_no'], ascending = False, inplace = True)

sns.barplot(xval['country'].head(10),y = xval['suicides_no'].head(10), data= xval)
plt.title('Топ 10 стран с максимальным кол-вом суицидов ', fontsize = 19)
plt.xlabel('Название страны')
plt.xticks(rotation = 90)
plt.ylabel('Кол-во самоубийств')
plt.rcParams['font.size'] = 19
plt.show()

### Можно выдвинуть гипотезу, что в больших странах люди совершают суицид чаще.
Возможно это связано с дорогой жизнью в таких странах, постоянного давления в больших городах, частого стресса.

Построим точечный график и проверим, связаны ли данные.

In [ ]:
fig = px.scatter(x= data['suicides_no'], y=data['population'])

fig.show()

По графику можно заметить, что данные связаны только в правой части графика, но дальше идет расхождение. Вычислим коэффициет корреляции.

In [ ]:
corr = data['suicides_no'].corr(data['population'])
print("Коэф. корреляции между населением и кол-ву суицидов : {0:.2f}".format(corr))

Данный коэффициент лежит в интервале от 0,5-0,7 и сигнализирует о связи.
Поэтому можно предположить что, чем больше популяция страны, тем больше совершается суицидов.

Получим характеристики по возрастным группам и кол-ву суицидов для дальнейшего анализа.

In [ ]:
data.groupby(['sex', 'age'])['suicides_no'].agg(['std', 'mean', 'var', 'median', 'min', 'max', ]).sort_index().reset_index()

Построим график, отражающий соотношене пола и кол-ва самоубийств по возрастным группам.

In [ ]:
sns.barplot(x = "sex", y = "suicides_no", data = data, hue = "age").set_title("Соотношение пола и кол-ва самоубийств")

Можно заметить следующее
 * Мужчины имеют более высокий уровень самоубийств по сравнению с женщинами.
 * Чем выше возраст, тем выше уровень самоубийств

Обратимся к показателям.

In [ ]:
# Датасет, только с мужчинами.
men_df = data.loc[data['sex'] == "male"].groupby('age')[['suicides_no']].sum().sort_index().reset_index()
# Датасет, только с женщинами.
women_df = data.loc[data['sex'] == "female"].groupby('age')[['suicides_no']].sum().sort_index().reset_index()

print("Общее кол-во суицидов мужчин", men_df[['suicides_no']].agg('sum'))
print("Общее кол-во суицидов женщин", women_df[['suicides_no']].agg('sum'))

### Можно заметить, что мужчины совершают суицид чаще женщин 6104173 > 1894294

In [ ]:
# Таблица для мужчин
men_df

In [ ]:
# Таблица для мужчин
women_df

Для визуализации построим точечный график по возрастным группам.

In [ ]:
g = sns.FacetGrid(data , row = 'sex',col = 'age',margin_titles=True)
g.map(plt.scatter,"suicides_no",'population',edgecolor = 'w')
plt.show()

### Проверим гипотезу, что среднее кол-во самоубийств возрастных групп зависит от пола.

Сначала исследуем возрастную группу 25-35 лет. Найдем корреляцию и для мужчин и для женщин по кол-ву суицидов и населению.

In [ ]:
sel_women = data.loc[data['sex'] == "female"]
sel_women = sel_women.loc[sel_women['age'] == '25-34 years']

sel_men = data.loc[data['sex'] == "male"]
sel_men = sel_men.loc[sel_men['age'] == '25-34 years']

g = sns.FacetGrid(sel_men , row = 'sex',col = 'age',margin_titles=True)
g.map(plt.scatter,"suicides_no",'population',edgecolor = 'w')
plt.show()

g = sns.FacetGrid(sel_women , row = 'sex',col = 'age',margin_titles=True)
g.map(plt.scatter,"suicides_no",'population',edgecolor = 'w')
plt.show()

corr = sel_men['suicides_no'].corr(sel_men['population'])
print("Коэф. корреляции между населением и кол-ву суицидов мужчин для группы 25-34: {0:.2f}".format(corr))

corr = sel_women['suicides_no'].corr(sel_women['population'])
print("Коэф. корреляции между населением и кол-ву суицидов женщин для группы 25-34: {0:.2f}".format(corr))

По графику у женщин более кучное распределение, поэтому и коэффициент более близкий к 1.
Однако и у мужчин и у женщин высокая положительная корреляция.

In [ ]:
women_rang_df = data.loc[data['sex'] == "female"]
men_rang_df = data.loc[data['sex'] == "male"]
# [1] means
# Возрастные группы для мужчин и женщин.
women_15_24 = women_rang_df['suicides_no'].loc[women_rang_df['age'] == '15-24 years']
women_25_34 = women_rang_df['suicides_no'].loc[women_rang_df['age'] == '25-34 years']
women_35_54 = women_rang_df['suicides_no'].loc[women_rang_df['age'] == '35-54 years']
men_15_24 = men_rang_df['suicides_no'].loc[men_rang_df['age'] == '15-24 years']
men_25_34 = men_rang_df['suicides_no'].loc[men_rang_df['age'] == '25-34 years']
men_35_54 = men_rang_df['suicides_no'].loc[men_rang_df['age'] == '35-54 years']
# Средние значения для возрастных групп
women_15_24_mean = women_15_24.mean()
women_25_34_mean = women_25_34.mean()
women_35_54_mean = women_35_54.mean()
men_15_24_mean = men_15_24.mean()
men_25_34_mean = men_25_34.mean()
men_35_54_mean = men_35_54.mean()
# Cуммы средних для пола
women_mean_sum = women_15_24_mean + women_25_34_mean + women_35_54_mean;
men_mean_sum = men_15_24_mean + men_25_34_mean + men_35_54_mean;
sex_mean_sum = women_mean_sum + men_mean_sum;
# Суммы средних для возраста
group_15_24_mean_sum = women_15_24_mean + men_15_24_mean;
group_25_34_mean_sum = women_25_34_mean + men_25_34_mean;
group_35_54_mean_sum = women_35_54_mean + men_35_54_mean;
group_mean_sum = group_15_24_mean_sum + group_25_34_mean_sum + group_35_54_mean_sum;
# [2] teoretical means
# Средние теоретические значения для возрастных групп
t_women_15_24_mean = (group_15_24_mean_sum * women_mean_sum) / sex_mean_sum;
t_women_25_34_mean = (group_25_34_mean_sum * women_mean_sum) / sex_mean_sum;
t_women_35_54_mean = (group_35_54_mean_sum * women_mean_sum) / sex_mean_sum;
t_men_15_24_mean = (group_15_24_mean_sum * men_mean_sum) / sex_mean_sum;
t_men_25_34_mean = (group_25_34_mean_sum * men_mean_sum) / sex_mean_sum;
t_men_35_54_mean = (group_35_54_mean_sum * men_mean_sum) / sex_mean_sum;
# Cуммы средних теоретических для пола
t_women_mean_sum = t_women_15_24_mean + t_women_25_34_mean + t_women_35_54_mean;
t_men_mean_sum = t_men_15_24_mean + t_men_25_34_mean + t_men_35_54_mean;
t_sex_mean_sum = t_women_mean_sum + t_men_mean_sum;
# Суммы средних теоретических для возраста
t_group_15_24_mean_sum = t_men_15_24_mean + t_women_15_24_mean;
t_group_25_34_mean_sum = t_men_25_34_mean + t_women_25_34_mean;
t_group_35_54_mean_sum = t_men_35_54_mean + t_women_35_54_mean;
t_group_mean_sum = t_group_15_24_mean_sum + t_group_25_34_mean_sum + t_group_35_54_mean_sum;
# Найдем X^2, v, X^2 крит.
val_15_24_women = ((women_15_24_mean - t_women_15_24_mean) ** 2 / t_women_15_24_mean);
val_25_34_women = ((women_25_34_mean - t_women_25_34_mean) ** 2 / t_women_25_34_mean);
val_35_54_women = ((women_35_54_mean - t_women_35_54_mean) ** 2 / t_women_35_54_mean);
val_15_24_men = ((men_15_24_mean - t_men_15_24_mean) ** 2 / t_men_15_24_mean);
val_25_34_men = ((men_25_34_mean - t_men_25_34_mean) ** 2 / t_men_25_34_mean);
val_35_54_men = ((men_35_54_mean - t_men_35_54_mean) ** 2 / t_men_35_54_mean);
# Найдем многофункциональный критерий X^2
x_2 = val_15_24_women + val_25_34_women + val_35_54_women + val_15_24_men + val_25_34_men + val_35_54_men;
f_2 = x_2 / group_mean_sum;
print('x^2: ',x_2);
print('Ф^2: ',f_2);
# Найдем v
v = (3 - 1) * (2 - 1);
print('v: ',v);
# Найдем критерий X^2.
x_2_krit = 5.991;  # (0,05)
print('x^2 krit: ',x_2_krit);
print('x^2 < x^2 krit H0 гипотеза подтверждается. Разница кол-ва суицидов возрастной группы случайны по полу.');

Давайте посмотрим, как соотносятся друг с другом объекты нашего набора данных.

In [ ]:
dic = {
      0:'5-14 years',
      1:'15-24 years',
      2:'25-34 years',
      3:'35-54 years',
      4:'55-74 years',
      5:'75+ years'
      }

age_int = {
            '5-14 years':0,
            '15-24 years':1,
            '25-34 years':2,
            '35-54 years':3,
            '55-74 years':4,
            '75+ years':5
          }

plot_aged_df = data

plot_aged_df['age_int_type'] = plot_aged_df['age'].map(age_int)

suicide = plot_aged_df.groupby('age_int_type')[['suicides_no']]

plt.figure(figsize=(10,5))
suicide.sum().plot(color = 'blue',figsize=(14, 5), grid = True)
sns.barplot(x=suicide.sum().index.map(dic.get),y=suicide.sum().suicides_no, alpha=0.7)
plt.show()

Мы видим, что возраст самоубийств имеет более или менее нормальное распределение. а большинство самоубийств происходят в возрасте 35-54 лет

До 35-54 лет количество самоубийств растет, после идет на спад.
Наблюдяется резкие скачки по самоубийствам между группами '5-14'-'15-24' и '25-34'-'35-54'

### Давайте посмотрим, как соотносятся друг с другом объекты нашего набора данных.

In [ ]:
agemap = {}
i = 0
for x in data.age.unique():
    agemap[x] = i
    i+=1

gendermap = {}
i = 0
for x in data.sex.unique():
    gendermap[x] = i
    i+=1

for_corr_plot_df = data

for_corr_plot_df['age_id'] = data['age'].map(agemap)
for_corr_plot_df['sex_id'] = data['sex'].map(gendermap)

x = for_corr_plot_df.drop(['sex','age'], axis = 1)
x = pd.get_dummies(x)
y = x[['suicides_no']]
x = x.drop('suicides_no',axis=1)

corr = for_corr_plot_df.corr()
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, vmax=1, cmap=cmap, center=0, square=True, linewidths=.5)

Население и кол-во самоубийств имеют связь, что было уже доказано выше. sex_id и suicides_no коррелируютДавайте теперь посмотрим, коррелируют ли самоубийства мужчин с самоубийствами женщин.

Удалим вспомогательные колонки из прошлого датафрейма.

In [ ]:
data = data.drop(['age_int_type', 'age_id', 'sex_id'], axis = 1)

Теперь, когда мы разделили самоубийства мужчин и женщин, давайте посмотрим, связаны ли они друг с другом.

In [ ]:
agemap = {}
i = 0
for x in data.age.unique():
    agemap[x] = i
    i+=1

gendermap = {}
i = 0
for x in data.sex.unique():
    gendermap[x] = i
    i+=1

data_for_corr = data
    
data_for_corr['age_id'] = data['age'].map(agemap)
data_for_corr['sex_id'] = data['sex'].map(gendermap)

x = data_for_corr.drop(['sex','age'], axis = 1)
x = pd.get_dummies(x)
y = x[['suicides_no']]
x = x.drop('suicides_no',axis=1)

corr = data_for_corr.corr()
cmap = sns.diverging_palette(220, 10, as_cmap=True)

data_by_gender = data_for_corr.loc[data_for_corr['sex_id'] == 1]
data_by_females = data_for_corr.loc[data_for_corr['sex_id'] == 0]

data_by_gender.rename(columns={'suicides_no':'m_suicides_no'}, inplace=True)
data_by_gender.rename(columns={'population':'m_population'}, inplace=True)

data_by_gender = data_by_gender.merge(data_by_females, on=['country', 'year', 'age'])
data_by_gender = data_by_gender.drop(labels=['age','age_id_y','sex_x','sex_y','sex_id_y','sex_id_x'],axis=1)
data_by_gender.rename(columns={'population':'f_population', 'age_id_x': 'age_id'}, inplace=True)
data_by_gender.rename(columns={'suicides_no':'f_suicides_no'}, inplace=True)

corr_by_gender = data_by_gender.corr()
sns.heatmap(corr_by_gender, vmax=1, cmap=cmap, center=0, square=True, linewidths=.5)

In [ ]:
data = data.drop(['age_id', 'sex_id'], axis = 1)

Получим числовые значения корреляция возрастной группы и кол-ва суицидов от разделенных датасетов по признаку пола.

In [ ]:
numeric_df = data
numeric_df = numeric_df.drop(['country'],axis = 1)

# Меняем возрастные категории и пол для возможности найти корреляцию
numeric_df['age'] = numeric_df['age'].replace('5-14 years',0)
numeric_df['age'] = numeric_df['age'].replace('15-24 years',1)
numeric_df['age'] = numeric_df['age'].replace('25-34 years',2)
numeric_df['age'] = numeric_df['age'].replace('35-54 years',3)
numeric_df['age'] = numeric_df['age'].replace('55-74 years',4)
numeric_df['age'] = numeric_df['age'].replace('75+ years',5)
numeric_df['sex'] = numeric_df['sex'].replace('female',0)
numeric_df['sex'] = numeric_df['sex'].replace('male',1)

# Датасет, только с мужчинами.
men_df = numeric_df.loc[numeric_df['sex'] == 1].groupby('age')[['suicides_no']].sum().sort_index().reset_index()

# Датасет, только с женщинами.
women_df = numeric_df.loc[numeric_df['sex'] == 0].groupby('age')[['suicides_no']].sum().sort_index().reset_index()

# теперь можно скорелировать суициды во возрастным группам мужчин и женщин

corr = men_df['suicides_no'].corr(men_df['age'])
print("Коэф.корреляции по возрасту и кол-ву суицидов для мужчин: {0:.2f}".format(corr))
corr = women_df['suicides_no'].corr(women_df['age'])
print("Коэф.корреляции по возрасту и кол-ву суицидов для женщин: {0:.2f}".format(corr))

В случае женщин видно, что связь сильнее.

### Проверим гипотезу, о зависимости между кол-вом суицидов и полом по странам.

Возьмем 3 страны для доказательства: Россия, Испания и Словения.

In [ ]:
# Выделяем 3 датафрейма по наблюдаемым странам.
sln_df = data.loc[data['country'] == 'Slovenia']
rus_df = data.loc[data['country'] == 'Russian Federation']
sp_df = data.loc[data['country'] == 'Spain']
# simple [1]
# Каждый датафрейм делим по полам и находим кол-во суицидов.
men_sln_sum = sln_df.loc[sln_df['sex'] == 'male']['suicides_no'].sum();
women_sln_sum = sln_df.loc[sln_df['sex'] == 'female']['suicides_no'].sum();
men_rus_sum = rus_df.loc[rus_df['sex'] == 'male']['suicides_no'].sum();
women_rus_sum = rus_df.loc[rus_df['sex'] == 'female']['suicides_no'].sum();
men_sp_sum = sp_df.loc[sp_df['sex'] == 'male']['suicides_no'].sum();
women_sp_sum = sp_df.loc[sp_df['sex'] == 'female']['suicides_no'].sum();
# Суммы суицидов по странам.
all_sln_sum =  men_sln_sum + women_sln_sum;
all_rus_sum =  men_rus_sum + women_rus_sum;
all_sp_sum =  men_sp_sum + women_sp_sum;
# Суммы суицидов по полам.
all_men_sum = men_sln_sum + men_rus_sum + men_sp_sum;
all_women_sum = women_sln_sum + women_rus_sum + women_sp_sum;
# Сумма суицидов всех рассматриваемых стран
all_country_suicide_sum = all_sln_sum + all_rus_sum + all_sp_sum;
# teoretical [2]
# Теоретическое кол-во самоубийств для мужчин и женщин.
t_men_sln_sum = (all_men_sum * all_sln_sum) / all_country_suicide_sum;
t_men_rus_sum = (all_men_sum * all_rus_sum) / all_country_suicide_sum;
t_men_sp_sum = (all_men_sum * all_sp_sum) / all_country_suicide_sum;
t_women_sln_sum = (all_women_sum * all_sln_sum) / all_country_suicide_sum;
t_women_rus_sum = (all_women_sum * all_rus_sum) / all_country_suicide_sum;
t_women_sp_sum = (all_women_sum * all_sp_sum) / all_country_suicide_sum;
# Теоретические суммы суицидов по странам.
t_all_sln_sum =  t_men_sln_sum + t_women_sln_sum;
t_all_rus_sum =  t_men_rus_sum + t_women_rus_sum;
t_all_sp_sum =  t_men_sp_sum + t_women_sp_sum;
# Теоретические суммы суицидов по полам.
t_all_men_sum = t_men_sln_sum + t_men_rus_sum + t_men_sp_sum;
t_all_women_sum = t_women_sln_sum + t_women_rus_sum + t_women_sp_sum;
# Найдем X^2, v, X^2 крит.
val_sln_men = ((men_sln_sum - t_men_sln_sum) ** 2 / t_men_sln_sum);
val_rus_men = ((men_rus_sum - t_men_rus_sum) ** 2 / t_men_rus_sum);
val_sp_men = ((men_sp_sum - t_men_sp_sum) ** 2 / t_men_sp_sum);
val_sln_women = ((women_sln_sum - t_women_sln_sum) ** 2 / t_women_sln_sum);
val_rus_women = ((women_rus_sum - t_women_rus_sum) ** 2 / t_women_rus_sum);
val_sp_women = ((women_sp_sum - t_women_sp_sum) ** 2 / t_women_sp_sum);
# Найдем X^2
x_2 = val_sln_men + val_rus_men + val_sp_men + val_sln_women + val_rus_women + val_sp_women;
f_2 = x_2 / all_country_suicide_sum;
# Найдем V
v = (2 - 1) * (3 - 1);
x_2_krit = 5.991;
print('x^2: ',x_2);
print('Ф^2: ',f_2);
print('v: ',v);
print('x^2 krit: ',x_2_krit);
print('x^2 > x^2 krit H0 гипотеза не подтверждается. Разница в кол-ве суицидов между полами не случайна по странам.');

### Проверим гипотезу, о связи между годом и кол-вом суицидов по полу.

In [ ]:
# Выделяем 3 датафрейма по наблюдаемым годам.
y_2011_df = data.loc[data['year'] == 2011]
y_2012_df = data.loc[data['year'] == 2012]
y_2013_df = data.loc[data['year'] == 2013]
# simple [1]
# Каждый датафрейм делим по полам и находим кол-во суицидов.
men_2011_sum = y_2011_df.loc[y_2011_df['sex'] == 'male']['suicides_no'].sum();
women_2011_sum = y_2011_df.loc[y_2011_df['sex'] == 'female']['suicides_no'].sum();
men_2012_sum = y_2012_df.loc[y_2012_df['sex'] == 'male']['suicides_no'].sum();
women_2012_sum = y_2012_df.loc[y_2012_df['sex'] == 'female']['suicides_no'].sum();
men_2013_sum = y_2013_df.loc[y_2013_df['sex'] == 'male']['suicides_no'].sum();
women_2013_sum = y_2013_df.loc[y_2013_df['sex'] == 'female']['suicides_no'].sum();
# Суммы суицидов по годам.
all_2011_sum =  men_2011_sum + women_2011_sum;
all_2012_sum =  men_2012_sum + women_2012_sum;
all_2013_sum =  men_2013_sum + women_2013_sum;
# Суммы суицидов по полу.
all_men_sum = men_2011_sum + men_2012_sum + men_2013_sum;
all_women_sum = women_2011_sum + women_2012_sum + women_2013_sum;
# Сумма суицидов всех рассматриваемых годов.
all_y_suicide_sum = all_2011_sum + all_2012_sum + all_2013_sum;
# teoretical [2]
# Теоретическое кол-во самоубийств для мужчин и женщин.
t_men_2011_sum = (all_men_sum * all_2011_sum) / all_y_suicide_sum;
t_men_2012_sum = (all_men_sum * all_2012_sum) / all_y_suicide_sum;
t_men_2013_sum = (all_men_sum * all_2013_sum) / all_y_suicide_sum;
t_women_2011_sum = (all_women_sum * all_2011_sum) / all_y_suicide_sum;
t_women_2012_sum = (all_women_sum * all_2012_sum) / all_y_suicide_sum;
t_women_2013_sum = (all_women_sum * all_2013_sum) / all_y_suicide_sum;
# Теоретические суммы суицидов по годам.
t_all_2011_sum =  t_men_2011_sum + t_women_2011_sum;
t_all_2012_sum =  t_men_2012_sum + t_women_2012_sum;
t_all_2013_sum =  t_men_2013_sum + t_women_2013_sum;
# Теоретические суммы суицидов по полу.
t_all_men_sum = t_men_2011_sum + t_men_2012_sum + t_men_2013_sum;
t_all_women_sum = t_women_2011_sum + t_women_2012_sum + t_women_2013_sum;
# Найдем X^2, v, X^2 крит.
val_2011_men = ((men_2011_sum - t_men_2011_sum) ** 2 / t_men_2011_sum);
val_2012_men = ((men_2012_sum - t_men_2012_sum) ** 2 / t_men_2012_sum);
val_2013_men = ((men_2013_sum - t_men_2013_sum) ** 2 / t_men_2013_sum);
val_2011_women = ((women_2011_sum - t_women_2011_sum) ** 2 / t_women_2011_sum);
val_2012_women = ((women_2012_sum - t_women_2012_sum) ** 2 / t_women_2012_sum);
val_2013_women = ((women_2013_sum - t_women_2013_sum) ** 2 / t_women_2013_sum);
# Найдем X^2
x_2 = val_2011_men + val_2012_men + val_2013_men + val_2011_women + val_2012_women + val_2013_women; 
f_2 = x_2 / all_y_suicide_sum;
# Найдем V
v = (2 - 1) * (3 - 1);
x_2_krit = 5.991;
print('x^2: ',x_2);
print('Ф^2: ',f_2);
print('v: ',v);
print('x^2 krit: ',x_2_krit);
print('x^2 < x^2 krit H0 гипотеза подтверждается. Разница в кол-ве суицидов между полами случайна по годам.');